In [1]:
import spacy
import whapi

In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
article = whapi.random_article()
article

5595249

In [4]:
search_results = whapi.search('cook', 500)
search_results

[{'title': 'Cook', 'article_id': 9673, 'url': 'https://www.wikihow.com/Cook'},
 {'title': 'Cook Pasta',
  'article_id': 2365,
  'url': 'https://www.wikihow.com/Cook-Pasta'},
 {'title': 'Cook Rice',
  'article_id': 43786,
  'url': 'https://www.wikihow.com/Cook-Rice'},
 {'title': 'Cook Chicken',
  'article_id': 382569,
  'url': 'https://www.wikihow.com/Cook-Chicken'},
 {'title': 'Cook Bacon',
  'article_id': 194774,
  'url': 'https://www.wikihow.com/Cook-Bacon'},
 {'title': 'Cook an Omelette',
  'article_id': 27976,
  'url': 'https://www.wikihow.com/Cook-an-Omelette'},
 {'title': 'Cook Lasagne',
  'article_id': 27718,
  'url': 'https://www.wikihow.com/Cook-Lasagne'},
 {'title': 'Cook Grasshoppers',
  'article_id': 662529,
  'url': 'https://www.wikihow.com/Cook-Grasshoppers'},
 {'title': 'Cook Sausages',
  'article_id': 252107,
  'url': 'https://www.wikihow.com/Cook-Sausages'},
 {'title': 'Cook Noodles',
  'article_id': 597282,
  'url': 'https://www.wikihow.com/Cook-Noodles'},
 {'title': 

In [5]:
# start out with some example text where it should find something
all_text = "Use a knife to cut the apple. Use the table saw to cut the counter top. Some more words here. "

In [6]:
#all_text = ""

count = 0

for result in search_results:
    print()
    article_info = whapi.return_details(result['article_id'])
    #print(article_info)
    try:
        steps = whapi.parse_steps(result['article_id'])
        for step in steps:
            this_text = steps[step]['summary'] + ". " + steps[step]['description']
            #print(this_text)
            all_text += this_text
        print("Found steps!")
        count += 1
    except:
        print("-")

#print(all_text)


Found steps!

Found steps!

-

-

Found steps!

-

-

-

Found steps!

Found steps!

-

Found steps!

-

-

Found steps!

-

-

Found steps!

Found steps!

Found steps!

-

Found steps!

-

-

-

-

Found steps!

-

-

-

-

-

Found steps!

-

-

-

-

Found steps!

-

-

Found steps!

Found steps!

-

Found steps!

Found steps!

Found steps!

-

-

-

-

-

Found steps!

Found steps!

Found steps!

Found steps!

-

-

Found steps!

Found steps!

-

-

-

-

Found steps!

-

-

-

-

-

-

-

-

Found steps!

-

-

-

Found steps!

Found steps!

Found steps!

Found steps!

-

-

Found steps!

Found steps!

Found steps!

-

-

-

-

-

-

-

Found steps!

-

-

-

Found steps!

Found steps!

Found steps!

Found steps!

Found steps!

-

Found steps!

Found steps!

-

Found steps!

-

Found steps!

-

-

-

-

Found steps!

Found steps!

Found steps!

-

Found steps!

-

-

-

-

Found steps!

-

Found steps!

Found steps!

-

-

-

-

-

-

Found steps!

Found steps!

Found steps!

-



In [7]:
print(count)

190


In [8]:
doc = nlp(all_text)
print(len(doc))

56578


In [22]:
from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

# match "use the ... to ... the ..." with anything inbetween
pattern1 = [
    {"LOWER": "use"}, 
    {"LOWER": {"IN": ["the", "a", "an"]}},
    {"IS_ALPHA": True, "OP": "+"},
    {"LOWER": "to"},
    {"IS_ALPHA": True, "OP": "+"},
    {"LOWER": {"IN": ["the", "a", "an"]}}, #, "OP": "?"},
    {"IS_PUNCT": False, "OP": "+"},
]

# match:
# "use the [... includes noun(s) but no verb(s)] 
#       to [... includes verb(s) but no noun(s)]
#    [the] [... until noun(s)]"
# -> does not allow any punctuation inside the match 
pattern2 = [
    {"LOWER": "use"},											#0
    {"LOWER": {"IN": ["the", "a", "an"]}},						#1
    {"IS_PUNCT": False, "POS": {"NOT_IN": ["VERB"]}, "OP": "*"},								#2
    {"POS": "NOUN", "OP": "+"},									#3
    {"IS_PUNCT": False, "POS": {"NOT_IN": ["VERB"]}, "OP": "*"},								#4
    {"LOWER": "to"},											#5
    {"IS_PUNCT": False, "POS": {"NOT_IN": ["NOUN"]}, "OP": "*"},								#6
    {"POS": "VERB", "OP": "+"},									#7
    {"IS_PUNCT": False, "POS": {"NOT_IN": ["NOUN"]}, "OP": "*"},								#8
    {"LOWER": {"IN": ["the", "a", "an"]}}, #, "OP": "?"},		#9
    {"IS_PUNCT": False, "POS": {"NOT_IN": ["VERB"]}, "OP": "*"},								#10
    {"POS": "NOUN", "OP": "+"},									#11
]

matcher.add("use", [pattern2])

In [23]:
matches = matcher(doc, with_alignments=True)
for match in matches:
    
    print("---")
    print()
    print(match) # (match_id, start_index, end_index, list of found constituents - numbers as indicated above)
    print(doc[match[1]:match[2]])
    print()
    
    start = match[1]
    end = match[2]
    constituents = match[3]
    
    tool_start = start + match[3].index(3)
    if 2 in match[3]:
        tool_start = start + match[3].index(2)
    tool_end = start + match[3].index(5)
    
    action_start = start + match[3].index(7)
    if 6 in match[3]:
        action_start = start + match[3].index(6)
    action_end = start + match[3].index(9)
    
    item_start = start + match[3].index(11)
    if 10 in match[3]:
        item_start = start + match[3].index(10)
    
    tool = doc[tool_start:tool_end]
    action = doc[action_start:action_end]
    item = doc[item_start:end]
    
    print(action)
    print(item)
    print(tool)

---

(6873750497785110593, 0, 7, [0, 1, 3, 5, 7, 9, 11])
Use a knife to cut the apple

cut
apple
knife
---

(6873750497785110593, 2969, 2978, [0, 1, 2, 2, 3, 5, 7, 9, 11])
use a small metal colander to keep the tails

keep
tails
small metal colander
---

(6873750497785110593, 4354, 4363, [0, 1, 2, 3, 5, 7, 9, 10, 11])
Use a kitchen knife to remove the blue membrane

remove
blue membrane
kitchen knife
---

(6873750497785110593, 6316, 6324, [0, 1, 3, 5, 7, 8, 9, 11])
use a knife to cut off the beard

cut off
beard
knife
---

(6873750497785110593, 10843, 10850, [0, 1, 3, 5, 7, 9, 11])
use a cookbook to find a recipe

find
recipe
cookbook
---

(6873750497785110593, 13158, 13168, [0, 1, 2, 3, 5, 7, 8, 9, 10, 11])
use a sharp knife to cut off the bottom stem

cut off
bottom stem
sharp knife
---

(6873750497785110593, 13212, 13223, [0, 1, 2, 3, 5, 7, 8, 8, 8, 9, 11])
Use a sharp knife to cut it off at an angle

cut it off at
angle
sharp knife
---

(6873750497785110593, 14344, 14355, [0, 1, 2,

In [24]:
print(doc[0:100])

Use a knife to cut the apple. Use the table saw to cut the counter top. Some more words here. Use fresh ingredients whenever possible, and don't forget to wash the vegetables beforehand. When shopping for fresh produce or meats, be certain to take the time to ensure that the texture, colors, and quality of the food you buy is the best in the batch. You should also try to stick to cooking with foods that are currently in season, as that is when they taste their best.
